In [ ]:
#GENERATIVE ADVERSERIAL NETWORK

#-DEFINE PROBLEM STATEMENT
#-load the data(with transforms and normalization)
   #-denormalize for visual inspection and of samples
    
#-DEFINE DISCRIMINATOR NETWORK
   #-study the activation functions:leaky relu
    
#-DEFINE GENERATOR NETWORK
   #-explain the output generator function 
    #-look at some sample output
    
#-DEFINE LOSSES,OPTIMIZERS AND HELPER FUNCTIONS FOR TRAINING
   #-for discriminator
    #-for generator
    
#-TRAIN THE MODEL
   #-save intermediate generated images to file
    
#LOOK AT SOME OUTPUTS

#-SAVE THE MODEL

In [ ]:
#we begin by downloading and importing the data as a pytorch adataset using the MNISThelper class from torchvision.dataset


In [1]:
import torch
import torchvision
from torchvision.transforms import ToTensor,Normalize,Compose
from torchvision.datasets import MNIST



In [2]:
mnist = MNIST(root='data',train=True,download=True,transform=Compose([ToTensor(),Normalize(mean=(0.5,),
                                                                                            std=(0.5))]))

RuntimeError: Dataset not found. You can use download=True to download it

In [ ]:
#note that we are transfering the pixel values from the range [0,1] to [-1,1].The reason for doing this will become clear when
#we define the generator network

In [ ]:
img,label = mnist[0]
print(label)
print(img[:,10:15,10:15])
torch.min(img),torch.max(img)


In [ ]:
#as expected, the pixel values range from -1 to 1. lets define a helper function to denormalize and view the image

In [ ]:
def denorm(x):
    out=(x+x)/2
    return out.clamp(0,1)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

img_norm = denorm(img)
plt.imshow(img_norm[0],cmap='gray')
print(label)

In [ ]:
from torch.utils.data import DataLoader
batch_size=100
data_loader = DataLoader(mnist,batch_size,shuffle=True)

In [ ]:
for img_batch,label_batch in data_loader:
    print('first batch')
    print(img_batch.shape)
    plt.imshow(img_batch[0][0],cmap='gray')
    print(label_batch)
    break

In [ ]:
device = torch.device('cuda',if torch.cuda.is_available() else 'cpu')

In [ ]:
#DISCRIMINATOR NETWORK MODEL

In [ ]:
import torch.nn as nn
image_size = 784
hidden_size = 256

In [ ]:
D = nn.Sequential(nn.Linear(image_size,hidden_size),
                  nn.LeakyRelu(0.2)
                  nn.Linaer(hidden_size,hidden_size)
                  nn.LeakyRelu(0.2)
                  nn.Linear(hidden_size,1)
                  nn.Sigmoid())
                  
                  

In [ ]:
D.to(device)

In [ ]:
#in the leakyrelu activation function we multiply any value less(negative) than zero
#by the LeakyRelu parameter, whereas in the Relu activation function, we put any negative value to zero

In [ ]:
#GENERATOR NETWORK/MODEL

In [ ]:
latent_size=64
G = nn.Sequential(
    nn.Linear(latent_size,hidden_size),
    nn.Relu(),
    nn.Linear(hidden_size,hidden_size),
    nn.Relu(),
    nn.Linear(hidden_size,image_size)
    #we use the Tanh activation function for the output of the generator
    nn.Tanh())

In [ ]:
Y = G(torch.randn(2,latent_size))
# convrert a two image vector to a 28x28 pixels
y.reshape(-1,28,28).shape


In [ ]:
gen_img=denorm(y.reshape((-1,28,28).detach())
               #detach return the neutral value without gradients etc;

In [ ]:
plt.imshow(gen_img[0],cmap='gray')

In [ ]:
#as expected the image is very noisy because we have no trained it yet

In [ ]:
#DISCRIMINATOR MODEL

#since the discriminator is a binary classification model,we can use the binary cross entropy loss function from the nn.BCELoss
#to quantify how well it is able to differentiate between real and generated images

In [ ]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(),lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(),lr=0.0002)


In [ ]:
#lets define a helper function to reset gradients and train the discriminator

In [ ]:
#resetting gradients back to zero can be done at the beginning of training
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    
def train_discriminator(images):
    #create the labels which are later used as inputs for the BCELoss
    real_labels=torch.ones(batch_size,1).to(device)
    fake_labels=torch.zeros(batch_size,1).to(device)
    
    #loss for real images
    output = D(images)
    d_loss_real=criterion(outputs,real_labels)
    real_score = outputs
    
    #loss for fake images
    #we generate 100 latent vectors to feed into the generator model and move them to the device
    #this is because we are not using the MNIST dataset here but rather generating random images ourselves 
    z = torch.randn(batch_size,latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs,fake_labels)
    fake_score = outputs
    
    #combine losses
    d_loss = d_loss_real + d_loss_fake
    #reset gradient
    reset_grad()
    #compute gradient for loss
    d_loss.backward()
    #adjust the parameters using backprop
    d_optimizer.step()
    

In [ ]:
#GENERATOR TRAINING

#since the outputs of the generator are vectors(which can be transformed to images),it is not obvious how we
#can train the generator.Since we know that the output images are fake,we can pass them into the discriminator,and compare the 
#output of the discriminator with the ground truth(ie. all fake) and use that to calculate the loss for the generator. In other
#words we can use the discriminator itself as part of the loss function

In [ ]:
def train_generator():
   # generate the fake images and calculate loss
z = torch.randn(batch_size,latent_size,1).to(device)
fake images = G(z)
outputs = D(fake_images)
#the generator should fool the discriminator to think that the images the generator produces is real.
#we therefore used torch.ones in this scenario,although the output images are fake 
#and should have an output of zero.This is so that the discriminator outputs will be closer to 1,thinking that the 
#fake images are actually real
labels = torch.ones(batch_size,1).to(device)
g_loss = criterion(outputs,labels)
#backprop and optimize
reset_grad()
g_loss.backward()
g_optimizer.step()
return g_loss,fake_images


In [ ]:
#TRAINING THE MODEL

#lets create a dictionary to where we can save intermediate outputs from the output to visually
#inspect the progress of the model.

In [ ]:
import os

sample_dir = 'samples'
if not os.path.exists(sample_dir)
os.makedir(sample_dir)

In [ ]:
#lets save a batch or real images that we can use for visual comparism while looking at the generated images

In [ ]:
from IPYTHON.display import image
from torchvision.utils import save_image

In [ ]:
#save some real images
for images,_ in data_loader:
    images = images.reshape(images.size(0),1,28,28)
    save_image(denorm(images),os.path.join(sample_dir,'real_images.png'),nrow=10)
    #the nrow tarameter is the number of images in each row of a grid data.It allows you to create a grid of images
    break

In [ ]:
#the image class from IPYTHON is used to view the image
image(os.path.join(sample_dir,'real_image.png'))

In [ ]:
#we now define a helper function to save a batch of generated images to disk at the end of every epoch
#we will use a fixed set of input vectors to the generator to see how the individual generated images 
#evolve over time as we train the model

In [ ]:
sample_vectors = torch.randn(batch_size,latent_size).to(device)

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images =fake_images.reshape(fake_images.size(0),1,28,28)
    fake_fname='fake_images-(0:0-4d).png'.format(index)
    print('saving',fake_fname)
    save_image(denorm(fake_images),os.path.join(sample_dir,fake_fname),nrow=10)

In [ ]:
#before training
#since multiple fake images are produced,the index zero(0) here indicates the first image
save_fake_images(0)
image(os.path.join(sample_dir,'fake_images-0000.png')

In [ ]:
#as expected, the images are very noisy
#we are now ready to train the model

In [ ]:
num_epochs = 50
total_step=len(data_loader)
d_losses,g_losses,real_scores,fake_scores=[],[],[],[]

for epoch in range(num_epochs):
    for i,(images,_) in enumerate(data_loader):
        #load a batch and transform to vectors
        images = images.reshape(batch_size,-1).to(device)
        #train discriminator and generator
        d_loss,real_score,fake_score=train_discriminator(images)
        g_loss,fake_images=train_generator()
        #inspect the losses
        if (i+1)%200=0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('epoch [{}/{}], step [{}/{}], d_losses:{:.4f}, g_losses:{:.4f}, D(x):{:.4f},
                  .format(epoch,num_epochs,i+1,total_step,d_loss.item(),g_loss.item(),real_score.mean().item()),
                  fake_score.mean().item()))
    #sample and save images
    save_fake_images(epoch+1)
            
            

